## Importação de Bibliotecas
---

In [2]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Primeiro Dataset

Nesta primeira parte vamos tratar do nosso primeiro DataSet que é o de `filmes`, contendo cerca de 5K de filmes.

---
## Visuliazação da Base de Dados de 5000 Filmes do IMDB
---

In [4]:
filmes = pd.read_csv('tmdb_5000_movies.csv')

In [5]:
filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


## Análise Exploratória dos Dados
---

Nesta etapa vamos analisar como os dados estão distribuidos, quais features serão significativas, tratar os valores que forem necessários.

In [7]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

### Exclusão de **Features Desnecessárias** para nosso algoritmo
---

Como a **Recomendação** ira levar em consideração o conteúdo do filme, informações como *língua, duração e etc* não será util.

In [9]:
filmes.drop(columns = ['budget', 'homepage', 'id', 'original_title', 'original_language',
                       'popularity', 'production_countries', 'release_date',
                       'revenue', 'runtime', 'spoken_languages',
                       'status', 'vote_average', 'vote_count'],
                        axis=1, inplace=True)

### Visualizção de Valores `NaN` e Únicos
---

In [11]:
faltantes = pd.DataFrame({'colunas': filmes.columns, 
                      'tipo': filmes.dtypes,
                      'Qtde valores NaN': filmes.isna().sum(),
                      '% valores NaN': (filmes.isna().sum()/filmes.shape[0]*100).round(2),
                      'valores únicos por feature': filmes.nunique(),
                       'valores duplicados': filmes.duplicated().sum()  
                         })
faltantes = faltantes.reset_index()
faltantes

,index,colunas,tipo,Qtde valores NaN,% valores NaN,valores únicos por feature,valores duplicados
0,genres,genres,object,0,0.00,1175,0
1,keywords,keywords,object,0,0.00,4222,0
2,overview,overview,object,3,0.06,4800,0
3,production_companies,production_companies,object,0,0.00,3697,0
4,tagline,tagline,object,844,17.57,3944,0
5,title,title,object,0,0.00,4800,0


#### Preenchimento da **Feature** `tagline`
---

Após a análise é possível notar que temos muitos dados `Nan` na coluna `tagline`, **não podemos dropar pois iremos perder muitos filmes *(844)* cerca de 17% da nossa base**, então vamos preenche - los com um espaço vazio aqueles que são `Nan` e mais pra frente no projeto isso não fara diferença.

In [13]:
filmes.tagline = filmes.tagline.fillna(' ')

#### Preenchimento da **Feature** `overview`
---

A **Feature** `overview`, possui apenas 3 valores `NaN` mas como estamos tentando preservar ao máximo nossa base iremos fazer a mesma coisa que fizemos com a `tagline` e no decorrer do desenvolvimento irá ficar claro o porque.

In [15]:
filmes.overview = filmes.overview.fillna(' ')

In [16]:
filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   genres                4803 non-null   object
 1   keywords              4803 non-null   object
 2   overview              4803 non-null   object
 3   production_companies  4803 non-null   object
 4   tagline               4803 non-null   object
 5   title                 4803 non-null   object
dtypes: object(6)
memory usage: 225.3+ KB


### Função para extrair infos das `features dentro de uma lista de dicionários`

----

Podemos notar que algumas de nossas **Features** estão dentro de uma lista de dicionários e os elementos que queremos está dentro da chave `'name'`

In [18]:
filmes.head()

,genres,keywords,overview,production_companies,tagline,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",A Plan No One Escapes,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",The Legend Ends,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","Lost in our world, found in another.",John Carter


E os elementos que queremos está dentro da chave `'name'`, como pode - se ver nos prints abaixo.

In [20]:
print(filmes['keywords'].iloc[0])
print('-------------------------')
print(filmes['genres'].iloc[0])
print('-------------------------')
print(filmes['production_companies'].iloc[0])


[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]
-------------------------
[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]
-------------------------
[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century 

#### **Função** `extrair_chave_lista`

---
Função para percorrer a lista de dicionários e retornar apenas os elementos da chave `name`, em uma lista, separados por `,`

---

In [22]:
def extrair_chave_lista(x, chave='name'):
    """
    Extrai os valores de uma chave de uma lista de dicionários armazenada como string.

    Retorna:
    - Uma lista com os valores, ou None se vazio ou erro.
    """
    try:
        lista = ast.literal_eval(x)
        if isinstance(lista, list) and len(lista) > 0:
            valores = [item.get(chave) for item in lista if chave in item]
            return valores 
        else:
            return []
    except:
        return []

In [23]:
filmes['genres'] = filmes['genres'].apply(extrair_chave_lista)

In [24]:
filmes['keywords'] = filmes['keywords'].apply(extrair_chave_lista)

In [25]:
filmes['production_companies'] = filmes['production_companies'].apply(extrair_chave_lista)

In [26]:
filmes.head()

,genres,keywords,overview,production_companies,tagline,title
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...",Enter the World of Pandora.,Avatar
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq, B24]",A Plan No One Escapes,Spectre
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,"[Legendary Pictures, Warner Bros., DC Entertai...",The Legend Ends,The Dark Knight Rises
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",[Walt Disney Pictures],"Lost in our world, found in another.",John Carter


# Segundo Dataset

---
## Visualização da Base de Dados `credits`, nela contém as informações sobre elenco e produção dos filmes.

In [28]:
credits = pd.read_csv('tmdb_5000_credits.csv')

In [29]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Análise Exploratória dos Dados
---

Nesta etapa vamos analisar como os dados estão distribuidos, quais features serão significativas, tratar os valores que forem necessários.

In [31]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


### Exclusão de **Features Desnecessárias** para nosso algoritmo
---

Como a **Recomendação** ira levar em consideração o conteúdo do filme, não será preciso ter a coluna `movie_id`

In [33]:
credits.drop('movie_id', axis=1, inplace=True)

### Função para extrair infos das `features dentro de uma lista de dicionários`

----

Como já sabemos que algumas informações importantes para nosso algoritmo esta dentro de uma **lista de dicionários**, vamos printar um exemplo e ver o que será util.

In [35]:
print(credits['cast'].iloc[0])
print('-------------------------')
print(credits['crew'].iloc[0])
print('-------------------------')

[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 2

#### Função `extrair_chave_lista_n`
---
Após análise é possível notar que a chave `name` da **Feature** contém os nomes dos atores qua atuaram, mas para não adicionarmos muita informção que não seja relevante vamos limitar a função para retornar apenas os 3 primeiros atores, que serão os principais de cada filme. 

In [37]:
def extrair_chave_lista_n(x, chave='name', n=3):
    """
    Extrai até n valores de uma chave de uma lista de dicionários armazenada como string.

    Retorna:
    - Uma lista com os valores, ou None se vazio ou erro.
    """
    try:
        lista = ast.literal_eval(x)
        if isinstance(lista, list) and len(lista) > 0:
            valores = [item[chave] for item in lista[:n] if chave in item]
            return valores 
        else:
            return []
    except:
        return []

In [38]:
credits['cast'] = credits['cast'].apply(extrair_chave_lista_n)

#### Função `extrair_nome_por_job`
---
Após examinar a **Feature** `crew`, é notável que ela armazena informações sobre a equipe responsável pelo desenvolvimento do filme, como Diretor, Editor e etc. Notamos que a chave responsávelpor armazenar estas funções é a `job` e assim vamos utilizar isso na nossa função para retornar o Diretor do filme.

In [40]:
def extrair_nome_por_job(x, job='Director'):
    """
    Extrai os nomes das pessoas com o job especificado de uma lista de dicionários armazenada como string.

    Retorna:
    - Uma lista com os nomes encontrados, ou None se não houver correspondências.
    """
    try:
        lista = ast.literal_eval(x)
        if isinstance(lista, list) and len(lista) > 0:
            nomes = [item['name'] for item in lista if item.get('job') == job]
            return nomes 
        else:
            return []
    except:
        return []

In [41]:
credits['director'] = credits['crew'].apply(extrair_nome_por_job)

In [42]:
credits.drop('crew',axis=1 ,inplace=True)

In [43]:
credits

,title,cast,director
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]
...,...,...,...
4798,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4799,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",[Edward Burns]
4800,"Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe]",[Scott Smith]
4801,Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton]",[Daniel Hsia]


# Terceiro Dataset

---
Agora que temos os dois datasets anteriores bem tratados podemos junta - los e criar um só armazenando tudo que precisamos `filmes2`.

In [45]:
filmes2 = filmes.merge(credits,on='title')

In [46]:
filmes2.head()

,genres,keywords,overview,production_companies,tagline,title,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...",Enter the World of Pandora.,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq, B24]",A Plan No One Escapes,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,"[Legendary Pictures, Warner Bros., DC Entertai...",The Legend Ends,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",[Walt Disney Pictures],"Lost in our world, found in another.",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


## Função `transformar_em_lista`

---

Após a união das tabelas foi possível notas que algumas colunas contém dados dentro de uma lista que foram aquelas colunas que trabalhamos acima, entretanto algumas colunas que contém informações relevantes como `tagline`, não está dentro de uma lista, isso dificulta nosso trabalho posteriormente para criarmos nossa principal **feature**, que é uma **sopa de palavras**. Vamos deixar todas colunas necessárias então dentro de uma lista.

In [48]:
def transformar_em_lista(valor):
    if isinstance(valor, list):
        return valor
    elif pd.isna(valor):  # trata NaN, None, etc.
        return []
    else:
        return [valor]

In [49]:
colunas = ['overview', 'tagline']

for col in colunas:
    filmes2[col] = filmes2[col].apply(transformar_em_lista)

In [50]:
filmes2.head()

,genres,keywords,overview,production_companies,tagline,title,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In the 22nd century, a paraplegic Marine is d...","[Ingenious Film Partners, Twentieth Century Fo...",[Enter the World of Pandora.],Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain Barbossa, long believed to be dead, h...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[At the end of the world, the adventure begins.]",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[A cryptic message from Bond’s past sends him ...,"[Columbia Pictures, Danjaq, B24]",[A Plan No One Escapes],Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",[Following the death of District Attorney Harv...,"[Legendary Pictures, Warner Bros., DC Entertai...",[The Legend Ends],The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[John Carter is a war-weary, former military c...",[Walt Disney Pictures],"[Lost in our world, found in another.]",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


## Criação da Feature `soup_of_words`

---

O que é a sopa de palavras?
A sopa de palavras é uma técnica usada em sistemas de recomendação baseados em conteúdo, onde combinamos diversas informações relevantes sobre um item (como um filme) em uma única string textual. Isso facilita quando precisamos passar por uma vetorização textual, que chegaremos em breve.

In [52]:
filmes2['soup_of_words'] = filmes2['genres'] + filmes2['keywords'] + filmes2['overview'] + filmes2['production_companies'] + filmes2['tagline'] + filmes2['cast'] + filmes2['director']

# Quarto Dataset

---

Agora chegamos no nosso quarto e último DataSet após um longo tratamento dos dados podemos separar apenas o que será necessário, `title` para saber o nome do filme e `soup_of_words` feature que será utilizada para fazer as recomendações.

In [54]:
filmes_final = filmes2[['title', 'soup_of_words']]

In [55]:
filmes_final

,title,soup_of_words
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drug abuse..."
2,Spectre,"[Action, Adventure, Crime, spy, based on novel..."
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,John Carter,"[Action, Adventure, Science Fiction, based on ..."
...,...,...
4804,El Mariachi,"[Action, Crime, Thriller, united states–mexico..."
4805,Newlyweds,"[Comedy, Romance, A newlywed couple's honeymoo..."
4806,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie, date, love ..."
4807,Shanghai Calling,[When ambitious New York attorney Sam is sent ...


## Retirar as informações de dentro da lista

---

Como usamos as listas afim apenas de faciltar na organização das Features e evitar erros ao unir todas Features em uma só, agora podemos aplicar uma função lambda para remover as listas.

In [57]:
filmes_final['soup_of_words'] = filmes_final['soup_of_words'].apply(lambda x:" ".join(x))

C:\Users\santo\AppData\Local\Temp\ipykernel_16400\2238889377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes_final['soup_of_words'] = filmes_final['soup_of_words'].apply(lambda x:" ".join(x))


In [58]:
filmes_final['soup_of_words'][0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Ingenious Film Partners Twentieth Century Fox Film Corporation Dune Entertainment Lightstorm Entertainment Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver James Cameron'

## Deixando todas palavras em *Minusculo*

---

Essa parte não é tão necessária mas afim de evitar erros por letras Maiusculas ou Minusculas, vamos deixar todas em um único padrão **Minusculas**.

In [60]:
filmes_final['soup_of_words'] = filmes_final['soup_of_words'].apply(lambda x:x.lower())

C:\Users\santo\AppData\Local\Temp\ipykernel_16400\2588239602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes_final['soup_of_words'] = filmes_final['soup_of_words'].apply(lambda x:x.lower())


In [61]:
filmes_final['soup_of_words'][0]

'action adventure fantasy science fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. ingenious film partners twentieth century fox film corporation dune entertainment lightstorm entertainment enter the world of pandora. sam worthington zoe saldana sigourney weaver james cameron'

## Vetorização de Palavras

 ---
 
Aqui estamos transformando o conteúdo textual da coluna `'soup_of_words'` em uma representação numérica
usando a técnica **TF-IDF (Term Frequency - Inverse Document Frequency).**

**O TfidfVectorizer realiza duas tarefas principais:**

1. Remove as "stop words" (palavras muito comuns em inglês como "the", "and", "is", etc.) que não agregam valor semântico.

2. Converte cada palavra em um peso numérico que representa sua importância em relação ao texto do filme e ao conjunto total de filmes.

O resultado é uma matriz onde cada linha representa um filme e cada coluna representa uma palavra.
Os valores nessa matriz indicam a relevância de cada palavra para cada filme, permitindo comparar filmes por similaridade de conteúdo.

In [63]:
vectorizer = TfidfVectorizer(stop_words='english')

In [64]:
tfidf = vectorizer.fit_transform(filmes_final['soup_of_words'])

## Cáculo da Similaridade de Cosseno

---

Após **transformar os textos em vetores com o TF-IDF**, usamos a função `cosine_similarity` para
medir o quão semelhantes são os filmes entre si com base no conteúdo textual.

**A similaridade do cosseno compara dois vetores e retorna um valor entre 0 e 1:**
- **1** indica máxima similaridade (mesma direção vetorial).
- **0** indica nenhuma similaridade (vetores ortogonais).

O resultado é uma matriz simétrica onde cada célula (i, j) representa a similaridade
entre o filme i e o filme j.

In [66]:
sim = cosine_similarity(tfidf)
sim

array([[1.        , 0.02210037, 0.01570498, ..., 0.01309062, 0.0201592 ,
        0.00342284],
       [0.02210037, 1.        , 0.01262518, ..., 0.01723875, 0.00887824,
        0.01129097],
       [0.01570498, 0.01262518, 1.        , ..., 0.00956417, 0.0431708 ,
        0.        ],
       ...,
       [0.01309062, 0.01723875, 0.00956417, ..., 1.        , 0.02212822,
        0.01547285],
       [0.0201592 , 0.00887824, 0.0431708 , ..., 0.02212822, 1.        ,
        0.0082123 ],
       [0.00342284, 0.01129097, 0.        , ..., 0.01547285, 0.0082123 ,
        1.        ]])

## Função de Recomendação `recomendar_filmes`

---

Esta função recebe o nome de um filme e retorna os 10 filmes mais similares, com base na **matriz de similaridade de cosseno**. O resultado é **retornado em formato de DataFrame, incluindo o título do filme e seu grau de similaridade.**

In [67]:
def recomendar_filmes(filme, df, matriz_similaridade):
    idx = df[df['title'] == filme].index[0]
    similaridades = list(enumerate(matriz_similaridade[idx]))
    similares_ordenados = sorted(similaridades, key=lambda x: x[1], reverse=True)[1:11]
    resultados = [(df.iloc[i].title, score) for i, score in similares_ordenados]
    
    return pd.DataFrame(resultados, columns=['Filme Recomendado', 'Similaridade'])

# Exemplo de uso:
# recomendar_filmes("The Matrix", filmes_final, similarity)

In [116]:
recomendar_filmes('Thor', filmes_final, sim)


,Filme Recomendado,Similaridade
0,Thor: The Dark World,0.555776
1,The Avengers,0.249398
2,Avengers: Age of Ultron,0.240287
3,Iron Man 2,0.226354
4,Captain America: Civil War,0.215434
5,Iron Man 3,0.212361
6,Captain America: The First Avenger,0.210953
7,Ant-Man,0.209403
8,Iron Man,0.204462
9,Guardians of the Galaxy,0.199113
